# Setting up the environment

## Importing required libraries:
Required libraries include:
<ul>
    <li><strong>Numpy</strong> and <strong>Pandas</strong> for data analysis</li>
    <li><strong>Matplotlib</strong> and <strong>Seaborn</strong> for data visualization</li>
</ul>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## Locating and opening the dataset
The dataset contains data files from 2009 up to 2018; for the purpose of this study, the latest one was chosen.

In [3]:
df = pd.read_csv("./dataset/2018.csv")
df.head

<bound method NDFrame.head of             FL_DATE OP_CARRIER  OP_CARRIER_FL_NUM ORIGIN DEST  CRS_DEP_TIME  \
0        2018-01-01         UA               2429    EWR  DEN          1517   
1        2018-01-01         UA               2427    LAS  SFO          1115   
2        2018-01-01         UA               2426    SNA  DEN          1335   
3        2018-01-01         UA               2425    RSW  ORD          1546   
4        2018-01-01         UA               2424    ORD  ALB           630   
...             ...        ...                ...    ...  ...           ...   
7213441  2018-12-31         AA               1815    DCA  CLT          1534   
7213442  2018-12-31         AA               1816    CLT  DFW          1751   
7213443  2018-12-31         AA               1817    CLT  MEM          2015   
7213444  2018-12-31         AA               1818    CLT  RDU          1300   
7213445  2018-12-31         AA               1818    RDU  CLT          1435   

         DEP_TIME  DE